In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from konlpy.tag import Okt
okt = Okt()

In [3]:
openFileName = '전자담배_naver_kin_903.txt'
    
f = open(openFileName, 'r', encoding='utf-8')
data = f.read()
f.close()

In [4]:
def tokenizer(raw, pos=["Noun"], stopword=[]): # 다른 품사 추가 원할 시 pos=["Noun", "Josa", ...]
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

documents1 = tokenizer(data)

In [5]:
import gensim

documents2 = [documents1] # gensim.corpora.Dictionary() 는 매개변수로 리스트 타입을 받기 때문에
dictionary = gensim.corpora.Dictionary(documents2)
print(dictionary.token2id)
print('dictionary size : %d' % len(dictionary)) # dictionary size : 37987

{'가가': 0, '가감': 1, '가게': 2, '가격': 3, '가경': 4, '가경동': 5, '가공': 6, '가구': 7, '가글': 8, '가기': 9, '가까이': 10, '가끔': 11, '가나': 12, '가늠': 13, '가능': 14, '가능성': 15, '가다가': 16, '가닥': 17, '가도': 18, '가득': 19, '가라': 20, '가래': 21, '가량': 22, '가려움증': 23, '가렵습니': 24, '가령': 25, '가로': 26, '가루': 27, '가맹': 28, '가면': 29, '가목': 30, '가물': 31, '가방': 32, '가변': 33, '가성': 34, '가세': 35, '가속': 36, '가스': 37, '가슴': 38, '가습기': 39, '가시': 40, '가신': 41, '가안': 42, '가야': 43, '가연': 44, '가연성': 45, '가열': 46, '가요': 47, '가용': 48, '가운데': 49, '가이드': 50, '가입': 51, '가장': 52, '가전': 53, '가정': 54, '가족': 55, '가죽': 56, '가중': 57, '가지': 58, '가짐': 59, '가창': 60, '가치': 61, '가치주': 62, '가친': 63, '가타': 64, '가트': 65, '가파도': 66, '가판': 67, '가품': 68, '가해': 69, '가형': 70, '가화': 71, '각각': 72, '각국': 73, '각기': 74, '각도': 75, '각별': 76, '각심': 77, '각오': 78, '각인': 79, '각자': 80, '각종': 81, '간격': 82, '간다': 83, '간만': 84, '간사이': 85, '간섭': 86, '간성': 87, '간식': 88, '간요': 89, '간접': 90, '간접흡연': 91, '간질': 92, '간판': 93, '간혹': 94, '갈고': 95, '갈색': 96, '갈수': 97, '갈수록': 98, '

In [6]:
corpus_path = '전자담배_naver_kin_903.txt'

class Documents:
    def __init__(self, path):
        self.path = path
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
            for doc in f:
                yield doc.strip().split()

documents = Documents(corpus_path)

In [7]:
class Corpus:
    def __init__(self, path, dictionary):
        self.path = path
        self.dictionary = dictionary
        self.length = 0
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
            for doc in f:
                yield self.dictionary.doc2bow(doc.split())
    def __len__(self):
        if self.length == 0:
            with open(self.path, encoding='utf-8') as f:
                for i, doc in enumerate(f):
                    continue
            self.length = i + 1
        return self.length

corpus = Corpus(corpus_path, dictionary)
for i, doc in enumerate(corpus):
    #if i >= 5: break
    print(doc)

[]
[]
[(316, 1), (505, 1), (509, 1), (769, 1), (1331, 3), (3355, 1), (4766, 1), (5066, 2)]
[]
[]
[(292, 2), (386, 1), (2422, 1), (3122, 1), (5026, 1), (5676, 3), (6628, 1)]
[]
[]
[(509, 1), (1240, 1), (2647, 1), (3355, 1), (4240, 1), (5035, 1), (5066, 2), (5166, 1), (6581, 1), (6793, 2)]
[]
[]
[(135, 1), (224, 1), (239, 2), (605, 1), (744, 1), (770, 1), (1143, 1), (1276, 1), (1331, 1), (1637, 1), (2117, 1), (2121, 1), (2254, 1), (2270, 2), (2521, 1), (2582, 1), (2780, 1), (2798, 1), (2916, 1), (3315, 1), (3696, 1), (3978, 1), (4554, 1), (4669, 1), (4744, 1), (5066, 1), (5324, 1), (6049, 1), (6328, 1), (6346, 1), (6433, 1), (6807, 1)]
[]
[]
[(646, 1), (903, 1), (1893, 1), (4601, 1), (5061, 2), (5066, 1), (5523, 1), (6421, 1), (6739, 1)]
[]
[]
[(863, 1), (3339, 1), (3664, 1), (3729, 1), (4351, 1), (5131, 1)]
[]
[]
[(316, 1), (855, 1), (1227, 2), (1251, 1), (1992, 1), (2780, 1), (4347, 1), (6430, 1), (6561, 1)]
[]
[]
[(81, 1), (316, 1), (834, 1), (5066, 2), (5610, 1)]
[]
[]
[(135, 1), (50

[(3, 1), (14, 2), (52, 2), (57, 1), (58, 1), (155, 1), (216, 1), (246, 1), (249, 1), (257, 1), (299, 1), (316, 4), (328, 1), (344, 1), (381, 1), (403, 1), (441, 1), (576, 1), (579, 1), (584, 1), (616, 1), (637, 1), (671, 2), (700, 1), (716, 1), (739, 1), (757, 1), (769, 3), (771, 1), (788, 2), (795, 1), (799, 2), (825, 3), (826, 1), (878, 1), (978, 1), (1003, 5), (1120, 4), (1122, 1), (1142, 3), (1152, 1), (1153, 1), (1188, 2), (1195, 4), (1202, 1), (1204, 3), (1227, 2), (1240, 2), (1264, 1), (1288, 4), (1331, 7), (1336, 1), (1435, 1), (1487, 2), (1561, 2), (1806, 4), (1880, 1), (1886, 1), (1892, 6), (1929, 5), (1987, 3), (1988, 1), (1989, 3), (2061, 4), (2102, 1), (2104, 2), (2121, 5), (2142, 1), (2155, 1), (2220, 3), (2242, 4), (2255, 1), (2257, 1), (2318, 2), (2365, 1), (2440, 1), (2445, 8), (2493, 2), (2531, 1), (2594, 1), (2680, 1), (2710, 2), (2746, 1), (2797, 1), (2895, 7), (2896, 1), (3047, 1), (3109, 1), (3126, 1), (3158, 1), (3225, 1), (3258, 3), (3267, 1), (3307, 1), (3324, 

[(5066, 1)]
[]
[]
[(13, 1), (46, 1), (52, 1), (58, 1), (69, 2), (210, 5), (246, 2), (257, 1), (265, 1), (381, 3), (392, 1), (487, 1), (671, 2), (675, 1), (769, 4), (788, 1), (825, 2), (904, 1), (994, 1), (1000, 1), (1003, 4), (1079, 1), (1122, 1), (1143, 1), (1188, 1), (1195, 2), (1199, 1), (1227, 1), (1261, 1), (1282, 2), (1331, 1), (1336, 1), (1387, 1), (1404, 1), (1409, 1), (1436, 1), (1529, 4), (1561, 2), (1581, 2), (1793, 1), (1806, 8), (1830, 2), (1842, 1), (1892, 3), (1893, 1), (1915, 1), (1929, 1), (1987, 1), (1988, 1), (1989, 1), (2061, 2), (2076, 3), (2093, 1), (2102, 1), (2104, 1), (2121, 2), (2139, 5), (2220, 2), (2255, 2), (2257, 1), (2292, 1), (2304, 1), (2318, 1), (2360, 2), (2441, 1), (2445, 2), (2493, 1), (2528, 1), (2679, 2), (2746, 1), (2858, 1), (2864, 1), (2895, 8), (2896, 1), (2989, 1), (3109, 1), (3134, 3), (3225, 1), (3307, 1), (3315, 1), (3320, 1), (3331, 2), (3365, 1), (3387, 3), (3419, 1), (3446, 2), (3626, 1), (3640, 2), (3688, 1), (3717, 1), (3729, 1), (375

[]
[]
[(671, 2), (3035, 1), (3134, 1), (4213, 1), (4268, 1), (4751, 1), (5066, 2), (5523, 1)]
[]
[]
[(3, 1), (51, 1), (52, 2), (163, 1), (270, 1), (352, 1), (381, 1), (523, 1), (566, 1), (591, 2), (671, 2), (769, 4), (1086, 1), (1143, 2), (1227, 2), (1432, 1), (1551, 1), (1708, 1), (1754, 1), (1806, 1), (1892, 1), (1979, 1), (1987, 1), (2121, 1), (2145, 1), (2220, 1), (2442, 1), (2629, 2), (2684, 1), (2895, 2), (2896, 1), (2951, 1), (3134, 2), (3355, 1), (3365, 1), (3380, 1), (3387, 1), (3468, 1), (3717, 1), (3833, 1), (3944, 1), (4268, 1), (4580, 1), (4618, 1), (4621, 1), (4685, 1), (4687, 1), (4884, 1), (5043, 1), (5066, 8), (5092, 1), (5131, 3), (5179, 1), (5208, 1), (5217, 4), (5356, 5), (5403, 1), (5456, 1), (5494, 1), (5533, 1), (5539, 1), (5567, 1), (5583, 1), (5610, 1), (5750, 1), (6463, 1), (6473, 1), (6495, 1), (6535, 1), (6823, 1), (6880, 1)]
[]
[]
[(1555, 2), (1984, 1), (4078, 1), (4699, 1), (4705, 1), (5066, 8), (5109, 1), (5268, 1), (5523, 1)]
[]
[]
[(316, 1), (769, 1), (

[]
[(3, 1), (14, 2), (52, 2), (57, 1), (58, 1), (155, 1), (246, 1), (249, 1), (257, 1), (299, 1), (316, 3), (381, 1), (441, 1), (572, 2), (579, 1), (584, 1), (700, 1), (739, 1), (757, 1), (769, 4), (788, 3), (799, 2), (825, 1), (878, 1), (913, 1), (978, 1), (989, 1), (1003, 3), (1120, 1), (1122, 1), (1142, 2), (1152, 1), (1153, 1), (1165, 1), (1188, 2), (1195, 2), (1202, 1), (1204, 2), (1288, 1), (1331, 6), (1336, 2), (1581, 1), (1806, 4), (1880, 1), (1886, 1), (1892, 7), (1929, 4), (1987, 2), (1988, 1), (1989, 2), (2061, 4), (2104, 2), (2121, 3), (2155, 1), (2242, 2), (2255, 1), (2318, 2), (2365, 1), (2441, 1), (2445, 7), (2493, 1), (2531, 1), (2680, 1), (2710, 1), (2797, 1), (2895, 4), (2896, 1), (3047, 1), (3109, 2), (3126, 1), (3158, 1), (3225, 1), (3258, 3), (3307, 1), (3354, 1), (3365, 1), (3387, 3), (3424, 1), (3446, 6), (3524, 1), (3579, 1), (3640, 3), (3696, 1), (3697, 2), (3706, 1), (3771, 2), (3833, 2), (4013, 2), (4065, 1), (4087, 6), (4118, 1), (4168, 2), (4186, 2), (4350,

[]
[(3, 1), (1105, 2), (1161, 2), (1551, 1), (1837, 1), (1987, 1), (2540, 1), (2607, 1), (3618, 1), (3729, 1), (5066, 2), (5092, 1), (5201, 1), (5249, 1), (5334, 1), (5718, 1), (5752, 1), (6581, 1)]
[]
[]
[(316, 1), (572, 1), (770, 1), (889, 1), (914, 1), (1194, 1), (1331, 1), (1334, 1), (1336, 1), (1581, 1), (2237, 1), (2270, 2), (2410, 1), (2441, 1), (2528, 1), (2962, 1), (3042, 1), (3696, 1), (3717, 1), (4078, 1), (4730, 1), (5023, 1), (5066, 1), (5093, 1), (5131, 1), (5494, 1), (5555, 1), (5583, 1), (5751, 1), (5999, 2), (6053, 1), (6527, 1), (6575, 1), (6596, 1), (6824, 1)]
[]
[]
[(200, 1), (257, 1), (572, 1), (769, 3), (826, 1), (827, 1), (913, 1), (993, 1), (1240, 1), (1317, 1), (1331, 1), (1336, 1), (1491, 1), (1561, 1), (1581, 1), (1644, 1), (1889, 1), (1892, 2), (2145, 1), (2157, 1), (2298, 2), (2441, 1), (2445, 1), (2520, 1), (2647, 1), (2680, 1), (2858, 1), (2888, 1), (2895, 4), (2970, 1), (3134, 1), (3324, 1), (3446, 1), (3696, 1), (3833, 1), (3993, 1), (4013, 1), (4181, 2

In [8]:
from gensim.models import LdaModel

lda_model = LdaModel(corpus, id2word=dictionary, num_topics=10)
print(lda_model)

LdaModel(num_terms=6902, num_topics=10, decay=0.5, chunksize=2000)


In [9]:
def get_topic_term_prob(lda_model):
    topic_term_freqs = lda_model.state.get_lambda()
    topic_term_prob = topic_term_freqs / topic_term_freqs.sum(axis=1)[:, None]
    return topic_term_prob

In [10]:
print(lda_model.alpha.shape) # (n_topics,)
print(lda_model.alpha.sum()) # 1.0

topic_term_prob = get_topic_term_prob(lda_model)
print(topic_term_prob.shape)     # (n_topics, n_terms)
print(topic_term_prob[0].sum())  # 1.0

(10,)
1.0
(10, 6902)
1.0


In [11]:
import pyLDAvis.gensim as gensimvis

prepared_data = gensimvis.prepare(lda_model, corpus, dictionary)

C:\Users\BOGYUNG\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [12]:
pyLDAvis.display(prepared_data)
# pyLDAvis.show(prepared_data)